# CS-VQE for Solving Electronic Structure

The second-quantised molecular Hailtonian takes the form

\begin{equation}
H = \sum_{pq} h_{pq} \hat{a}^*_p \hat{a}_q + \frac{1}{2} \sum_{pqrs} h_{pqrs} \hat{a}^*_p \hat{a}^*_q \hat{a}_r \hat{a}_s
\end{equation}

where $\hat{a}$ is the Fermionic annihiliation operator, with its adjoint $\hat{a}^*$ the creation operator. We may map this Hamiltonian onto qubits via some transformation such as Jordan-Wigner, where

\begin{equation}
\hat{a}_p \mapsto \frac{1}{2} (X_p + i Y_p) \otimes \bigotimes_{q<p} Z_q.
\end{equation}
Let us see what this Hamiltonian looks like in practice:

In [1]:
from symmer.chemistry import MoleculeBuilder

atoms  = ['Be']
coords = [(0,0,0)]
charge = 0
spin   = 0
basis  = 'sto-3g'

molecule = MoleculeBuilder(
    geometry=list(zip(atoms, coords)), 
    charge=charge, 
    basis=basis, 
    spin=spin, 
    run_fci=True, 
    print_info=True
)

OSError: dlopen(/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/pyscf/lib/libnp_helper.dylib, 0x0006): Library not loaded: '/usr/local/opt/libomp/lib/libomp.dylib'
  Referenced from: '/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/pyscf/lib/libnp_helper.dylib'
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)

In [ ]:
molecule.data_dictionary()

The second-quantised Hamiltonian for this molecular system:

In [ ]:
print(molecule.H)

... and after a mapping onto qubits, we obtain the Hamiltonian

In [ ]:
H = molecule.H_q
print(H)

Note that the ground state energy of this Hamiltonian matches the full-configuration interaction (FCI) energy above

In [ ]:
from symmer.utils import exact_gs_energy
from symmer.symplectic import QuantumState

gs_nrg, gs_psi = exact_gs_energy(molecule.H_q.to_sparse_matrix)

print(f'Ground state energy = {gs_nrg}, with FCI error {abs(gs_nrg - molecule.fci_energy)}.\n')
print('The ground state is:\n'); print(gs_psi)

# Taper the Hamiltonian first

Before we apply CS-VQE, one might as well perform qubit tapering since it does not introduce any systematic error

In [ ]:
from symmer.projection import QubitTapering

QT = QubitTapering(H)
print(f'Qubit tapering permits a reduction of {H.n_qubits} -> {H.n_qubits-QT.n_taper} qubits.\n')
print('The following symmetry generators were identified:\n')
print(QT.symmetry_generators); print()
print('which we may rotate onto the single-qubit Pauli operators\n') 
print(QT.symmetry_generators.rotate_onto_single_qubit_paulis()); print()
print('via a sequence of Clifford operations R_k = e^{i pi/4 P_k} where:\n')
for index, (P_k, angle) in enumerate(QT.symmetry_generators.stabilizer_rotations):
    P_k.sigfig=0
    print(f'P_{index} = {P_k}')

In [ ]:
H_taper = QT.taper_it(ref_state=molecule.hf_array) 
UCC_taper = QT.taper_it(aux_operator=molecule.UCC_q)
# note the reference state to place us in the correct symmetry sector!

gs_nrg_tap, gs_psi_tap = exact_gs_energy(H_taper.to_sparse_matrix)

print(f'The ground state energy of the Hamiltonian is {gs_nrg}')
print(f'and for the {H_taper.n_qubits}-qubit tapered Hamiltonian it is {gs_nrg_tap};')
print(f'the energy error is {abs(gs_nrg - gs_nrg_tap)}.\n')
print('The tapered ground state is:\n')
print(gs_psi_tap)

# Now we build the CS-VQE model

This involves partitioning $H$ into _noncontextual_ and _contextual_ componenents satisfying $H = H_\mathrm{noncon} + H_\mathrm{context}$.

In [ ]:
from symmer.projection import ContextualSubspace

cs_vqe = ContextualSubspace(H_taper, noncontextual_strategy='SingleSweep_magnitude')

Noncontextual Hamiltonians have a very specific form, namely their terms $\mathcal{T}$ may be decomposed as
\begin{equation}
    \mathcal{T} = \mathcal{S} \cup \mathcal{C}_1 \cup \dots \cup \mathcal{C}_M 
\end{equation}
where $\mathcal{S}$ contains the terms that commute globally (i.e. the noncontextual symmetries) and a collection of cliques with respect to commutation.

In [ ]:
cs_vqe.noncontextual_operator.decomposed

Representing the Hamiltonian as a graph with edges denoting commutation between Pauli operators, this structure corresponds with a partition into complete subgraphs:

In [ ]:
from symmer.symplectic import ObservableGraph, PauliwordOp
from matplotlib import pyplot as plt
import networkx as nx
import numpy as np

adjmat = cs_vqe.noncontextual_operator.adjacency_matrix.copy()
mask_nonsymmetries = ~np.all(adjmat, axis=1)
adjmat = adjmat[mask_nonsymmetries,:][:,mask_nonsymmetries]
np.fill_diagonal(adjmat, False)
graph = nx.from_numpy_matrix(adjmat)

plt.figure()
pos = nx.spring_layout(graph, k=1)
nx.draw_networkx_nodes(graph,pos,node_color='black', node_size=50)
nx.draw_networkx_edges(graph, pos,width=1)
plt.show()

Identifying a generating set $\mathcal{G}$ for the symmetry terms $\mathcal{S}$ and constructing the clique operator $A(\vec{r}) = \sum_{i=1}^M r_i C_i$ for clique representatives $C_i \in \mathcal{C}_i$ and coefficients $\vec{r} \in \mathbb{R}^M$ satisfying $|\vec{r}|=1$ allows us to rewrite

\begin{equation}
    H_\mathrm{noncon} = \sum_{P \in \overline{\mathcal{G}}} \bigg(h_{P}^\prime + \sum_{i=1}^M h_{P,i} C_i \bigg) P,
\end{equation}

and yields a classical objective function over parameters $\vec{\nu} \in \{\pm 1\}^{|\mathcal{G}|}$ and $\vec{r} \in \mathbb{R}^M$ for the noncontetual energy expectation value:

\begin{equation}\label{classical_objective}
\begin{aligned}
    \eta(\vec{\nu}, \vec{r}) 
    :={} & {\langle H_\mathrm{noncon} \rangle_{(\vec{\nu}, \vec{r})}} \\
    ={} & \sum_{P \in \overline{\mathcal{G}}} \bigg(h_{P}^\prime + \sum_{i=1}^M h_{P,i} \langle{C_i}\rangle_{(\vec{\nu}, \vec{r})} \bigg) \langle{P}\rangle_{(\vec{\nu}, \vec{r})} \\
    ={} & \sum_{P \in \overline{\mathcal{G}}} \bigg(h_{P}^\prime + \sum_{i=1}^M h_{P,i} r_i \bigg) \prod_{G \in \mathcal{G}_{P}} \nu_{f(G)}.
\end{aligned}
\end{equation}

In [ ]:
print('The symmetry generators G are:\n')
print(cs_vqe.noncontextual_operator.symmetry_generators); print()
print('The clique operator A(r) is:\n')
print(cs_vqe.noncontextual_operator.clique_operator); print()
print(f'The optimal paramters are '+ 
      f'nu={cs_vqe.noncontextual_operator.symmetry_generators.coeff_vec}, r={cs_vqe.noncontextual_operator.clique_operator.coeff_vec},')
print(f'which yields a noncontextual energy of n(nu,r) = {cs_vqe.noncontextual_operator.energy}')

# Quantum corrections

We have obtained a classical estimate to the ground state energy from the noncontextual Hamiltonian - the power of CS-VQE is in how it derives quantum corrections from a VQE simulation of the contextual Hamiltonian. To ensure the corrections are valid, we constrain the contextual problem by the solution to the noncontextual one. This is achieved via stabilizer subspace projections - similar in vain to tapering, except the imposed symmetries are those of the noncontextual Hamiltonian. This results in a loss of information, since symmetry-breaking terms vanish under such a projection. 

We choose a set of stabilizers $\mathcal{F}$ we would like to fix in the contextual subspace; identifying a unitary operation $U_\mathcal{F}$ mapping $\mathcal{F}$ to single-qubit Pauli operators we may obtain a reduced $(N-|\mathcal{F}|)$-qubit Hamiltonian

\begin{equation}
    \tilde{H}= \pi_{U_{\mathcal{F}}} (H_\mathrm{context})
\end{equation}

with the projection $\pi(\cdot)$ defined in [this](https://arxiv.org/pdf/2204.02150.pdf) paper.

In [ ]:
from symmer.symplectic import StabilizerOp

cs_vqe.update_stabilizers(n_qubits = 3, strategy='aux_preserving', aux_operator=UCC_taper)
# the user may specify stabilizers manually also:
# cs_vqe.manual_stabilizers(['ZIZZZ', 'ZZIII'])

H_cs = cs_vqe.project_onto_subspace()
print(f'We may project into the contextual subspace stabilized by '+
      f'{list(cs_vqe.stabilizers.to_dictionary.keys())} via a sequence of rotations:\n')

if cs_vqe.perform_unitary_partitioning:
    print(f'Note the clique operator')
    print(cs_vqe.noncontextual_operator.clique_operator) 
    print('is enforced for this set of stabilizers.')
#for index, (P_k, angle) in enumerate(
#    cs_vqe.unitary_partitioning_rotations+cs_vqe.stabilizers.stabilizer_rotations
#    ):
#    P_k.sigfig=0
#    print(f'P_{index} = {P_k}')
    
print(f'\nand tracing over qubit positions {cs_vqe.stab_qubit_indices} yields the {H_cs.n_qubits}-qubit Hamiltonian:\n')
print(H_cs)

In [ ]:
cs_vqe.stabilizers.stabilizer_rotations

In [ ]:
cs_vqe.unitary_partitioning_rotations

Performing a VQE simulation over this Hamiltonian yields the quantum-corrected energy

In [ ]:
from symmer.symplectic import ObservableOp, AnsatzOp

UCC_cs = cs_vqe.project_onto_subspace(UCC_taper)
obs_cs = ObservableOp(H_cs.symp_matrix, H_cs.coeff_vec)
anz_cs = AnsatzOp(UCC_cs.symp_matrix, np.random.random(UCC_cs.n_terms))
ref_cs = molecule.hf_array[QT.free_qubit_indices][cs_vqe.free_qubit_indices]

vqe_result, interim_data = obs_cs.VQE(
    ansatz_op=anz_cs, 
    ref_state=ref_cs,
    optimizer='BFGS',
    maxiter=20
)
print(f'Converged VQE energy = {vqe_result.fun} with FCI error {abs(vqe_result.fun-molecule.fci_energy)}')

In [ ]:
fig, axis = plt.subplots(figsize=(10,6))

X,Y = zip(*interim_data['values'])
axis.plot(X, abs(np.array(Y)-molecule.fci_energy), color='black', label='$E(\\theta)$')
axis.hlines(abs(cs_vqe.noncontextual_operator.energy-molecule.fci_energy), 1, X[-1], color='orange', ls='--', label='Noncontextual energy')
axis.fill_between(X, 0, 0.0016, color='green', alpha=0.2, label='Chemical accuracy')
axis.set_yscale('log')
axis.set_ylabel('Error w.r.t. FCI [Ha]')
axis.set_xlabel('nfev')
axis.legend()
plt.show()